<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/Makine_Egitim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

# --- 1. ADIM: VERİ ÜRETİMİ ---
n_samples = 20000
print(f"TEKNOFEST 2026 Üniversite Şartnamesine uygun {n_samples} veri üretiliyor...")

def generate_bio_data(n):
    sequences = []
    bio_features = []
    labels = []

    for _ in range(n):
        # 1. Sekans (11 harf)
        seq = ''.join(np.random.choice(list('ACGT'), size=11))

        # 2. Biyolojik Sayısal Veriler
        risk_score = np.random.beta(2, 2)
        maf = np.random.exponential(0.1)
        if maf > 0.5: maf = 0.5
        conservation = np.random.uniform(0, 10)
        hydrophobicity = np.random.uniform(-5, 5)

        # Etiketleme Mantığı
        score_calc = (risk_score * 0.5) + (conservation/10 * 0.3) + ((0.5 - maf)*2 * 0.2)
        score_calc += np.random.normal(0, 0.05)

        if score_calc > 0.60:
            labels.append(1) # Patojenik
        else:
            labels.append(0) # Benign

        sequences.append(seq)
        bio_features.append([risk_score, maf, conservation, hydrophobicity])

    return sequences, np.array(bio_features), np.array(labels)

sequences, X_numerical, y = generate_bio_data(n_samples)

# --- 2. ADIM: EĞİTİM VERİSİNİ TEXT OLARAK KAYDETME ---
print(">> Veriler 'egitim_veri_seti.txt' dosyasına kaydediliyor...")

# Verileri bir DataFrame'de toplayıp dosyaya yazıyoruz
df_ham_veri = pd.DataFrame(X_numerical, columns=['Risk_Skoru', 'MAF_Sikligi', 'Korunmusluk', 'Hidrofobiklik'])
df_ham_veri.insert(0, 'DNA_Dizisi', sequences) # En başa diziyi ekle
df_ham_veri['HEDEF_ETIKET'] = y # 1: Patojenik, 0: Benign

# Dosyaya yazma (Tab ile ayrılmış TXT)
df_ham_veri.to_csv('egitim_veri_seti.txt', sep='\t', index=False)
print(">> Kayıt Başarılı! (Dosya adı: egitim_veri_seti.txt)\n")

# --- 3. ADIM: ÖN İŞLEME VE MODEL HAZIRLIĞI ---
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(sequences)
X_seq = tokenizer.texts_to_sequences(sequences)
X_seq_pad = pad_sequences(X_seq, maxlen=11, padding='post')

X_num = X_numerical # Zaten sayısal

# Eğitim ve Test Ayrımı
X_seq_train, X_seq_test, X_num_train, X_num_test, y_train, y_test = train_test_split(
    X_seq_pad, X_num, y, test_size=0.2, random_state=42
)

# Model Mimarisi (Hybrid)
input_seq = Input(shape=(11,), name="Sequence_Input")
x1 = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=8, input_length=11)(input_seq)
x1 = Conv1D(filters=16, kernel_size=3, activation='relu')(x1)
x1 = GlobalMaxPooling1D()(x1)

input_num = Input(shape=(4,), name="Bio_Features_Input")
x2 = Dense(16, activation='relu')(input_num)
x2 = BatchNormalization()(x2)

combined = Concatenate()([x1, x2])
z = Dense(32, activation='relu')(combined)
z = Dropout(0.3)(z)
output = Dense(1, activation='sigmoid')(z)

model = Model(inputs=[input_seq, input_num], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# --- 4. ADIM: EĞİTİM ---
print("Model Eğitiliyor...")
model.fit([X_seq_train, X_num_train], y_train, epochs=8, batch_size=64, validation_split=0.1, verbose=1)

# --- 5. ADIM: TEST VE SONUÇLARI KAYDETME ---
print("\n>> Test işlemi yapılıyor ve sonuçlar kaydediliyor...")

# Tahminleri al
y_pred_prob = model.predict([X_seq_test, X_num_test])
y_pred = (y_pred_prob > 0.5).astype(int)

# F1 Skorunu Ekrana Bas
f1 = f1_score(y_test, y_pred)
print(f"Test F1 Skoru: {f1:.4f}")

# --- KAYIT İŞLEMİ (TEST SONUÇLARI) ---
# Test dizilerini (sayısal formattan) tekrar okunabilir harflere çevirelim
test_seqs_text = tokenizer.sequences_to_texts(X_seq_test)
# Tokenizer araya boşluk koyar (A C G T), onları siliyoruz (ACGT) ve büyük harf yapıyoruz
test_seqs_text = [s.replace(' ', '').upper() for s in test_seqs_text]

# Sonuç Tablosu Oluşturma
df_sonuc = pd.DataFrame(X_num_test, columns=['Risk_Skoru', 'MAF_Sikligi', 'Korunmusluk', 'Hidrofobiklik'])
df_sonuc.insert(0, 'Test_Edilen_DNA', test_seqs_text)
df_sonuc['Gercek_Etiket'] = y_test
df_sonuc['Tahmin_Olasiligi'] = y_pred_prob.flatten() # Virgüllü Olasılık
df_sonuc['Tahmin_Sinifi'] = y_pred.flatten()         # 0 veya 1

# Doğru bilip bilmediğini kontrol eden sütun
df_sonuc['SONUC_DURUMU'] = np.where(df_sonuc['Gercek_Etiket'] == df_sonuc['Tahmin_Sinifi'], 'DOGRU_BILDI', 'HATA_YAPTI')

# Dosyaya kaydetme
df_sonuc.to_csv('test_sonuc_raporu.txt', sep='\t', index=False)
print(">> Test sonuçları kaydedildi! (Dosya adı: test_sonuc_raporu.txt)")
print(">> Dosyayı açıp hangi veride hata yaptığını 'SONUC_DURUMU' sütunundan inceleyebilirsiniz.")